## Goal

The goal here is to be able to run PySpark for one off small analyses which do not require connecting to remote AWS instances. 

Everything is LOCAL. ON the notebook instance. TODO : confirm this.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages net.snowflake:snowflake-jdbc:3.6.24,net.snowflake:spark-snowflake_2.11:2.4.12-spark_2.3 pyspark-shell'

In [3]:
spark = SparkSession.builder.master('local').appName('test').config('spark.driver.memory', '5G').getOrCreate()

In [4]:
spark

In [5]:
spark.builder.config('spark.executor.memory', '16G')
spark.builder.config("spark.executor.cores", "4")

### Creating a Spark dataframe 

In [6]:
raw_df = spark.createDataFrame(
    [
        (100, 'Cengage'),
        (10, 'Boston')
    ],# the actual data
    ["Column1", "Column2"] #column names
)

raw_df.show()

+-------+-------+
|Column1|Column2|
+-------+-------+
|    100|Cengage|
|     10| Boston|
+-------+-------+



### Connection with Snowflake

In [7]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
sfOptions = {"sfURL":"*", "sfAccount":"*", "sfUser":"", "sfPassword":"*", "sfDatabase":"*", "sfSchema":"*", "sfWarehouse":"*"}


In [8]:
df = spark.read.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("query", "select * from dev.zsp.VALID_SUBS_DATALEAK").load()

In [9]:
df.show(5)

+--------------------+--------------------+--------------------+---------+--------------+--------------+-----------------------+------------------+-------------+
|       USER_SSO_GUID|  SUBSCRIPTION_START|    SUBSCRIPTION_END|LAST_DATE|REL_DAY_OF_SUB|SUB_CUTOFF_DAY|PREV_SUBSCRIPTION_STATE|SUBSCRIPTION_STATE|LATEST_RECORD|
+--------------------+--------------------+--------------------+---------+--------------+--------------+-----------------------+------------------+-------------+
|08063b485f58711d:...|-0400 2018-08-20 ...|-0400 2018-09-03 ...|     null|         -9999|            14|                   null|      trial_access|         true|
|95096707cdeb01a5:...|-0400 2018-08-22 ...|-0400 2018-09-05 ...|     null|         -9999|            14|                   null|      trial_access|         true|
|07bb9eddddf07ddc:...|-0400 2018-08-30 ...|-0400 2018-09-13 ...|     null|         -9999|            14|                   null|      trial_access|         true|
|409005c1e01f9b38:...|-0400 

In [11]:
spark.stop()